In [1]:
# Install required libraries (run in your environment if needed)
# !pip install faker reportlab

from faker import Faker
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from datetime import datetime, timedelta
import random
import os

# Ensure output directory exists
output_dir = "out"
os.makedirs(output_dir, exist_ok=True)

# Initialize Faker
fake = Faker()


PDF generated: out\chase_statement_classic.pdf
PDF generated: out\chase_statement_alternate.pdf
PDF generated: out\chase_statement_app_6611.pdf


In [ ]:

# Unified synthetic data generation
def generate_statement_data():
    """Generate synthetic data compatible with Classic, Alternate, and App styles"""
    start = datetime(2025, 6, 1)
    end = datetime(2025, 6, 30)
    statement_period = f"{start.strftime('%B %d, %Y')} - {end.strftime('%B %d, %Y')}"
    account_type = random.choice(["Chase Total Checking", "Chase Business Complete Checking"])
    account_holder = fake.name()
    account_holder_address_lines = [fake.street_address(), f"{fake.city()}, {fake.state_abbr()} {fake.zipcode()}"]
    full_account_number = fake.bban()
    last_four = full_account_number[-4:]
    
    # Generate deposits and withdrawals with varied descriptions
    deposit_descriptions = ["Direct Deposit", "Salary Deposit", "Online Transfer", "Cash Deposit"]
    withdrawal_descriptions = ["Payment", "Grocery Shopping", "Utility Payment", "Online Purchase"]
    deposits = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(deposit_descriptions)}",
            "amount": round(random.uniform(300, 5000), 2)
        } for _ in range(random.randint(2, 5))
    ]
    withdrawals = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(withdrawal_descriptions)}",
            "amount": -round(random.uniform(40, 1000), 2)
        } for _ in range(random.randint(3, 7))
    ]
    
    # Calculate summary
    beginning_balance = round(random.uniform(2000, 10000), 2)
    deposits_total = round(sum(t["amount"] for t in deposits), 2)
    withdrawals_total = round(sum(t["amount"] for t in withdrawals), 2)
    ending_balance = beginning_balance + deposits_total + withdrawals_total
    
    # Generate daily balances
    daily_balances = []
    current_balance = beginning_balance
    current_date = start
    while current_date <= end:
        date_str = current_date.strftime("%m/%d/%Y")
        daily_balances.append({
            "date": date_str,
            "description": random.choice(["Daily Balance", "End of Day Balance"]),
            "amount": current_balance
        })
        for deposit in deposits:
            if deposit["date"] == date_str:
                current_balance += deposit["amount"]
        for withdrawal in withdrawals:
            if withdrawal["date"] == date_str:
                current_balance += withdrawal["amount"]
        daily_balances[-1]["amount"] = round(current_balance, 2)
        current_date += timedelta(days=1)
    
    return {
        "logo": "sample_logos/chase_bank_logo.png",
        "account_type": account_type,
        "statement_period": statement_period,
        "statement_date": end.strftime("%B %d, %Y"),
        "account_number": full_account_number,
        "account_number_last_four": last_four,
        "account_holder": account_holder,
        "account_holder_address_lines": account_holder_address_lines,
        "account_holder_address": ", ".join(account_holder_address_lines),
        "deposits": deposits,
        "withdrawals": withdrawals,
        "daily_balances": daily_balances,
        "show_fee_waiver": random.choice([True, False]),
        "summary": {
            "beginning_balance": f"${beginning_balance:,.2f}",
            "deposits_count": len(deposits),
            "deposits_total": f"${deposits_total:,.2f}",
            "withdrawals_count": len(withdrawals),
            "withdrawals_total": f"${abs(withdrawals_total):,.2f}",
            "transactions_count": len(deposits) + len(withdrawals),
            "ending_balance": f"${ending_balance:,.2f}"
        }
    }

# Generate shared data
ctx = generate_statement_data()


In [5]:

# ---------------------------------------------------------------------------------
# Chase Classic Style
# ---------------------------------------------------------------------------------

def create_classic_statement(ctx):
    output_file = os.path.join(output_dir, "chase_statement_classic.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=40, rightMargin=40, topMargin=40, bottomMargin=40)
    elements = []

    # Styles
    styles = getSampleStyleSheet()
    normal_style = ParagraphStyle(
        name='Normal12', fontName='Helvetica', fontSize=12, leading=14.4, spaceAfter=4
    )
    bold_style = ParagraphStyle(
        name='Bold12', fontName='Helvetica-Bold', fontSize=12, leading=14.4, spaceAfter=4
    )
    header_style = ParagraphStyle(
        name='Header', fontName='Helvetica-Bold', fontSize=14, leading=16.8, spaceAfter=10, alignment=1
    )
    cs_header_style = ParagraphStyle(
        name='CSHeader', fontName='Helvetica-Bold', fontSize=12, leading=14.4, textTransform='uppercase'
    )
    footnote_style = ParagraphStyle(
        name='Footnote', fontName='Helvetica', fontSize=10, leading=12, spaceAfter=10
    )

    # Header
    PAGE_WIDTH, _ = letter
    LEFT_MARGIN = 40
    RIGHT_MARGIN = 40
    INNER_WIDTH = PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN

    left_block = [
        Paragraph("JPMorgan Chase Bank, N.A.", normal_style),
        Paragraph("PO Box 659754", normal_style),
        Paragraph("San Antonio, TX 78265-9754", normal_style)
    ]
    if os.path.exists(ctx['logo']):
        left_block.insert(0, Image(ctx['logo'], width=120, height=45))

    right_top = [
        Paragraph(ctx['statement_period'], normal_style),
        Paragraph(f"Account Number: {ctx['account_number']}", normal_style)
    ]
    cs_header = [Paragraph("CUSTOMER SERVICE INFORMATION", ParagraphStyle('CSHeaderCentered', parent=cs_header_style, alignment=1))]
    cs_rows = [
        ["Web site:", "chase.com"],
        ["Service Center:", "1-800-242-7338"],
        ["Hearing Impaired:", "1-800-242-7383"],
        ["Para Español:", "1-888-622-4273"],
        ["International Calls:", "1-713-262-1679"]
    ]
    cs_data = [cs_header] + cs_rows
    cs_table = Table(cs_data, colWidths=[1.9*inch, 1.9*inch])
    cs_table.setStyle(TableStyle([
        ('SPAN', (0,0), (-1,0)),
        ('LINEABOVE', (0,0), (-1,0), 3, colors.black),
        ('LINEBELOW', (0,0), (-1,0), 3, colors.black),
        ('ALIGN', (1,1), (1,-1), 'RIGHT'),
        ('FONTSIZE', (0,1), (-1,-1), 12),
        ('LEFTPADDING', (0,1), (0,-1), 0),
        ('RIGHTPADDING', (1,1), (1,-1), 0)
    ]))
    right_block = right_top + [Spacer(1, 0.15*inch), cs_table]

    header_table = Table(
        [[left_block, right_block]],
        colWidths=[0.60*INNER_WIDTH, 0.40*INNER_WIDTH],
        hAlign='LEFT'
    )
    header_table.setStyle(TableStyle([
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('ALIGN', (1,0), (1,0), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(header_table)
    elements.append(Spacer(1, 0.3*inch))

    # Payee Info
    elements.append(Paragraph(ctx['account_holder'], bold_style))
    for line in ctx['account_holder_address_lines']:
        elements.append(Paragraph(line, normal_style))
    elements.append(Spacer(1, 0.2*inch))

    # Important Account Information
    elements.append(Paragraph("Important Account Information", header_style))
    if ctx['account_type'] == "Chase Total Checking":
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Chase Total Checking accounts will increase to $15 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked Chase savings account with a balance of $5,000 or more.\n"
            "Starting June 30, 2025, Chase will introduce real-time transaction alerts for Chase Total Checking accounts via the Chase Mobile app to enhance account monitoring. Enable alerts at chase.com/alerts.\n"
            "Effective July 15, 2025, Chase will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Chase Total Checking accounts.\n"
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at <b>1-800-242-7338</b>, available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Chase Business Complete Checking accounts will increase to $20 unless you maintain a minimum daily balance of $2,000, have $2,000 in net purchases on a Chase Business Debit Card, or maintain linked Chase business accounts with a combined balance of $10,000.\n"
            "Starting June 30, 2025, Chase will offer enhanced cash flow tools for Chase Business Complete Checking accounts via Chase Online, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Chase will reduce wire transfer fees to $25 for domestic transfers for Chase Business Complete Checking accounts, down from $30.\n"
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at <b>1-800-242-7338</b>, available 24/7."
        )
    elements.append(Paragraph(info_text, normal_style))

    # Checking Summary
    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph(ctx['account_type'], bold_style))
    elements.append(Paragraph("Checking Summary", header_style))
    summary_data = [
        ["", "Instances", "Amount"],
        ["Beginning Balance", "–", ctx['summary']['beginning_balance']],
        ["Deposits and Additions", str(ctx['summary']['deposits_count']), ctx['summary']['deposits_total']],
        ["Electronic Withdrawals", str(ctx['summary']['withdrawals_count']), ctx['summary']['withdrawals_total']],
        ["Ending Balance", str(ctx['summary']['transactions_count']), ctx['summary']['ending_balance']]
    ]
    summary_table = Table(summary_data, colWidths=[2.4*inch, 1.2*inch, 1.2*inch])
    summary_table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('ALIGN', (2, 1), (2, -1), 'RIGHT'),
    ]))
    elements.append(summary_table)
    if ctx['show_fee_waiver']:
        fee_text = (
            "Your monthly service fee was waived because you maintained an average checking balance of $1,500 or had qualifying direct deposits totaling $500 or more during the statement period."
            if ctx['account_type'] == "Chase Total Checking"
            else "Your monthly service fee was waived because you maintained an average checking balance of $10,000 or had $2,500 in qualifying direct deposits during the statement period."
        )
        elements.append(Paragraph(fee_text, normal_style))

    # Deposits and Additions
    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph("Deposits and Additions", header_style))
    deposits_data = [["Date", "Description", "Amount"]]
    for deposit in ctx['deposits']:
        deposits_data.append([deposit["date"], deposit["description"], f"${deposit['amount']:,.2f}"])
    if not ctx['deposits']:
        deposits_data.append(["No deposits for this period.", "", ""])
    deposits_data.append(["Total Deposits and Additions", "", ctx['summary']['deposits_total']])
    deposits_table = Table(deposits_data, colWidths=[1.2*inch, 4.8*inch, 1.2*inch])
    deposits_table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('LINEBELOW', (0, 1), (-1, -2), 2, colors.black),
        ('ALIGN', (2, 1), (2, -1), 'RIGHT'),
        ('FONT', (0, -1), (-1, -1), 'Helvetica-Bold', 12),
    ]))
    elements.append(deposits_table)

    # Withdrawals
    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph("Withdrawals", header_style))
    withdrawals_data = [["Date", "Description", "Amount"]]
    for withdrawal in ctx['withdrawals']:
        withdrawals_data.append([withdrawal["date"], withdrawal["description"], f"${abs(withdrawal['amount']):,.2f}"])
    if not ctx['withdrawals']:
        withdrawals_data.append(["No withdrawals for this period.", "", ""])
    withdrawals_data.append(["Total Electronic Withdrawals", "", ctx['summary']['withdrawals_total']])
    withdrawals_table = Table(withdrawals_data, colWidths=[1.2*inch, 4.8*inch, 1.2*inch])
    withdrawals_table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('LINEBELOW', (0, 1), (-1, -2), 2, colors.black),
        ('ALIGN', (2, 1), (2, -1), 'RIGHT'),
        ('FONT', (0, -1), (-1, -1), 'Helvetica-Bold', 12),
    ]))
    elements.append(withdrawals_table)

    # Daily Ending Balance
    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph("Daily Ending Balance", header_style))
    balance_data = [["Date", "Amount"]]
    for balance in ctx['daily_balances']:
        balance_data.append([balance["date"], f"${balance['amount']:,.2f}"])
    balance_table = Table(balance_data, colWidths=[3*inch, 3*inch])
    balance_table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('ALIGN', (1, 1), (1, -1), 'RIGHT'),
    ]))
    elements.append(balance_table)

    # Footnotes
    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph("Disclosures", footnote_style))
    elements.append(Paragraph(
        "All account transactions are subject to the Chase Deposit Account Agreement, available at chase.com. "
        "Interest rates and Annual Percentage Yields (APYs) may change without notice. "
        "For details on overdraft policies and fees, visit chase.com/overdraft or call 1-800-242-7338.",
        footnote_style
    ))
    elements.append(Paragraph(
        "JPMorgan Chase Bank, N.A. is a Member FDIC. Equal Housing Lender.",
        footnote_style
    ))

    # Build PDF
    doc.build(elements)
    print(f"PDF generated: {output_file}")


In [7]:

# ---------------------------------------------------------------------------------
# Chase Alternate Style
# ---------------------------------------------------------------------------------

def create_alternate_statement(ctx):
    BLUE = colors.Color(0/255, 84/255, 166/255)
    GREY = colors.HexColor('#666666')
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle('Body', fontName='Helvetica', fontSize=10, leading=12))
    styles.add(ParagraphStyle('BodyBold', parent=styles['Body'], fontName='Helvetica-Bold'))
    styles.add(ParagraphStyle('SectionHead', fontName='Helvetica-Bold', fontSize=12, leading=14, textColor=BLUE,
                              spaceAfter=6, spaceBefore=18, borderWidth=1, borderColor=BLUE,
                              backColor=None, alignment=0, leftIndent=0, rightIndent=0, borderPadding=(4, 6, 4)))
    styles.add(ParagraphStyle('Foot', fontName='Helvetica', fontSize=8, leading=9.6))

    def money(x): return "${:,.2f}".format(x)

    def make_header(ctx):
        elems = []
        tbl = Table(
            [[
                [
                    Image(ctx['logo'], width=1.7*inch, height=0.5*inch) if os.path.exists(ctx['logo']) else '',
                    Paragraph("JPMorgan Chase Bank, N.A.", styles['BodyBold']),
                    Paragraph("PO Box 659754", styles['Body']),
                    Paragraph("San Antonio, TX 78265-9754", styles['Body']),
                ],
                '',
                [
                    Paragraph(ctx['statement_period'], styles['BodyBold']),
                    Paragraph(f"Account Number: {ctx['account_number']}", styles['Body']),
                    Spacer(1, 2),
                    make_cs_table()
                ]
            ]],
            colWidths=[3.1*inch, 0.3*inch, 3.1*inch]
        )
        tbl.setStyle(TableStyle([
            ('VALIGN', (0,0), (-1,-1), 'TOP'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ]))
        elems.append(tbl)
        return elems

    def make_cs_table():
        data = [
            [Paragraph("CUSTOMER SERVICE INFORMATION", styles['BodyBold']), ''],
            ["Web site:", "chase.com"],
            ["Service Center:", "1-800-242-7338"],
            ["Hearing Impaired:", "1-800-242-7383"],
            ["Para Español:", "1-888-622-4273"],
            ["International Calls:", "1-713-262-1679"],
        ]
        tbl = Table(data, colWidths=[1.65*inch, 1.4*inch])
        tbl.setStyle(TableStyle([
            ('SPAN', (0,0), (1,0)),
            ('ALIGN', (0,0), (1,0), 'CENTER'),
            ('FONTSIZE', (0,1), (-1,-1), 10),
            ('LINEABOVE', (0,0), (-1,0), 1, BLUE),
            ('LINEBELOW', (0,0), (-1,0), 1, BLUE),
            ('ALIGN', (1,1), (1,-1), 'RIGHT'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ]))
        return tbl

    def make_summary(ctx):
        s = ctx['summary']
        data = [
            ['', 'Instances', 'Amount'],
            ['Beginning Balance', '–', s['beginning_balance']],
            ['Deposits & Additions', s['deposits_count'], s['deposits_total']],
            ['Electronic Withdrawals', s['withdrawals_count'], s['withdrawals_total']],
            ['Ending Balance', s['transactions_count'], s['ending_balance']],
        ]
        tbl = Table(data, colWidths=[2.4*inch, 1.0*inch, 1.2*inch])
        tbl.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), BLUE),
            ('TEXTCOLOR', (0,0), (-1,0), colors.white),
            ('FONT', (0,0), (-1,0), 'Helvetica-Bold', 10),
            ('ALIGN', (1,1), (1,-1), 'CENTER'),
            ('ALIGN', (2,1), (2,-1), 'RIGHT'),
            ('GRID', (0,0), (-1,-1), 0.5, GREY),
            ('LEFTPADDING', (0,0), (-1,-1), 4),
            ('RIGHTPADDING', (0,0), (-1,-1), 4),
            ('TOPPADDING', (0,0), (-1,-1), 2),
            ('BOTTOMPADDING', (0,0), (-1,-1), 2),
        ]))
        return tbl

    def make_txn_table(label, rows, colWidths):
        data = [['Date', 'Description', 'Amount']]
        for r in rows:
            data.append([r['date'], r['description'], money(r['amount'])])
        total = money(sum(r['amount'] for r in rows))
        data.append([f'Total {label}', '', total])
        tbl = Table(data, colWidths=colWidths)
        tbl.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), BLUE),
            ('TEXTCOLOR', (0,0), (-1,0), colors.white),
            ('FONT', (0,0), (-1,0), 'Helvetica-Bold', 10),
            ('ALIGN', (2,1), (2,-1), 'RIGHT'),
            ('LINEBELOW', (0,1), (-1,-2), 0.5, GREY),
            ('FONT', (0,-1), (-1,-1), 'Helvetica-Bold', 10),
            ('LEFTPADDING', (0,0), (-1,-1), 4),
            ('RIGHTPADDING', (0,0), (-1,-1), 4),
            ('TOPPADDING', (0,0), (-1,-1), 2),
            ('BOTTOMPADDING', (0,0), (-1,-1), 2),
        ]))
        return tbl

    output_file = os.path.join(output_dir, "chase_statement_alternate.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=40, rightMargin=40, topMargin=40, bottomMargin=40)
    elements = []

    elements += make_header(ctx)
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(ctx['account_holder'], styles['BodyBold']))
    for line in ctx['account_holder_address_lines']:
        elements.append(Paragraph(line, styles['Body']))
    elements.append(Spacer(1, 12))

    elements.append(Paragraph("Important Account Information", styles['SectionHead']))
    if ctx['account_type'] == "Chase Total Checking":
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Chase Total Checking accounts will increase to $15 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked Chase savings account with a balance of $5,000 or more.\n"
            "Starting June 30, 2025, Chase will introduce real-time transaction alerts for Chase Total Checking accounts via the Chase Mobile app to enhance account monitoring. Enable alerts at chase.com/alerts.\n"
            "Effective July 15, 2025, Chase will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for Chase Total Checking accounts.\n"
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at <b>1-800-242-7338</b>, available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly service fee for Chase Business Complete Checking accounts will increase to $20 unless you maintain a minimum daily balance of $2,000, have $2,000 in net purchases on a Chase Business Debit Card, or maintain linked Chase business accounts with a combined balance of $10,000.\n"
            "Starting June 30, 2025, Chase will offer enhanced cash flow tools for Chase Business Complete Checking accounts via Chase Online, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Chase will reduce wire transfer fees to $25 for domestic transfers for Chase Business Complete Checking accounts, down from $30.\n"
            "For questions about your account or these changes, please visit chase.com or contact our Customer Service team at <b>1-800-242-7338</b>, available 24/7."
        )
    elements.append(Paragraph(info_text, styles['Body']))

    elements.append(Spacer(1, 0.4*inch))
    elements.append(Paragraph(ctx['account_type'], styles['BodyBold']))
    elements.append(make_summary(ctx))

    elements.append(Paragraph("Deposits and Additions", styles['SectionHead']))
    elements.append(make_txn_table("Deposits and Additions", ctx['deposits'], [1.0*inch, 3.7*inch, 1.2*inch]))

    elements.append(Paragraph("Withdrawals", styles['SectionHead']))
    elements.append(make_txn_table("Electronic Withdrawals", ctx['withdrawals'], [1.0*inch, 3.7*inch, 1.2*inch]))

    elements.append(Paragraph("Daily Ending Balance", styles['SectionHead']))
    balance_data = [["Date", "Amount"]]
    for balance in ctx['daily_balances']:
        balance_data.append([balance["date"], f"${balance['amount']:,.2f}"])
    balance_table = Table(balance_data, colWidths=[3*inch, 3*inch])
    balance_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), BLUE),
        ('TEXTCOLOR', (0,0), (-1,0), colors.white),
        ('FONT', (0,0), (-1,0), 'Helvetica-Bold', 10),
        ('ALIGN', (1,1), (1,-1), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 4),
        ('RIGHTPADDING', (0,0), (-1,-1), 4),
        ('TOPPADDING', (0,0), (-1,-1), 2),
        ('BOTTOMPADDING', (0,0), (-1,-1), 2),
    ]))
    elements.append(balance_table)

    elements.append(Spacer(1, 20))
    elements.append(Paragraph("Disclosures", styles['SectionHead']))
    elements.append(Paragraph(
        "All account transactions are subject to the Chase Deposit Account Agreement, available at chase.com. "
        "Interest rates and Annual Percentage Yields (APYs) may change without notice. "
        "For details on overdraft policies and fees, visit chase.com/overdraft or call 1-800-242-7338.",
        styles['Foot']
    ))
    elements.append(Paragraph(
        "JPMorgan Chase Bank, N.A. is a Member FDIC. Equal Housing Lender.",
        styles['Foot']
    ))

    doc.build(elements)
    print(f"PDF generated: {output_file}")


In [ ]:

# ---------------------------------------------------------------------------------
# Chase App Style
# ---------------------------------------------------------------------------------

def create_app_statement(ctx):
    def create_card_content(content, width):
        card_table = Table([[content]], colWidths=[width - 20])
        card_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, -1), colors.white),
            ('BOX', (0, 0), (-1, -1), 1, colors.HexColor("#e5e7eb")),
            ('INNERGRID', (0, 0), (-1, -1), 0, colors.transparent),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('LEFTPADDING', (0, 0), (-1, -1), 10),
            ('RIGHTPADDING', (0, 0), (-1, -1), 10),
            ('TOPPADDING', (0, 0), (-1, -1), 10),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 10),
        ]))
        return KeepTogether(card_table)

    def get_styles():
        styles = getSampleStyleSheet()
        return {
            "title": ParagraphStyle(name='Title', fontSize=18, textColor=colors.HexColor("#003087"), fontName="Helvetica-Bold", spaceAfter=10, leading=20),
            "normal": ParagraphStyle(name='Normal', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica", spaceAfter=5),
            "bold": ParagraphStyle(name='Bold', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica-Bold", spaceAfter=5),
            "blue_bold": ParagraphStyle(name='BlueBold', fontSize=10, textColor=colors.HexColor("#003087"), fontName="Helvetica-Bold", spaceAfter=5),
            "customer_service": ParagraphStyle(name='CustomerService', fontSize=10, textColor=colors.white, fontName="Helvetica-Bold", backColor=colors.HexColor("#003087"), spaceAfter=5, borderPadding=5, borderColor=colors.HexColor("#003087"), borderWidth=1, borderRadius=4),
            "green_text": ParagraphStyle(name='GreenText', fontSize=10, textColor=colors.HexColor("#10b981"), fontName="Helvetica-Bold", spaceAfter=5),
            "red_text": ParagraphStyle(name='RedText', fontSize=10, textColor=colors.HexColor("#ef4444"), fontName="Helvetica-Bold", spaceAfter=5),
            "small": ParagraphStyle(name='Small', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica", spaceAfter=5, leading=14.5),
            "small_summary": ParagraphStyle(name='SmallSummary', fontSize=12, fontName="Helvetica", spaceBefore=10)
        }

    def header_detailed(data, styles):
        content = []
        if os.path.exists(data['logo']):
            img = Image(data['logo'])
            if img._width and img._height:
                img.drawHeight = img._height * inch / 72
                img.drawWidth = img._width * inch / 72
            else:
                img.drawHeight = 0.5 * inch
                img.drawWidth = 1.5 * inch
            content.append(img)
        content.extend([
            Spacer(1, 0.1*inch),
            Paragraph("JPMorgan Chase Bank, N.A.", styles["bold"]),
            Paragraph("PO Box 659754, San Antonio, TX 78265-9754", styles["normal"]),
            Paragraph(f"Statement for: {data['account_holder']}", styles["normal"]),
        ])
        right_content = [
            Paragraph(f"Period: {data['statement_period']}", styles["blue_bold"]),
            Paragraph(f"Account Number: ****{data['account_number_last_four']}", styles["normal"]),
            Spacer(1, 0.1*inch),
            Paragraph("Customer Service", styles["customer_service"]),
            Paragraph("chase.com | 1-800-242-7338", styles["normal"]),
            Paragraph("Español: 1-888-622-4273 | International: 1-713-262-1679", styles["normal"])
        ]
        table = Table([[content, right_content]], colWidths=[3*inch, 3*inch])
        table.setStyle(TableStyle([
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
            ('BACKGROUND', (0, 0), (-1, -1), colors.white),
            ('LEFTPADDING', (0, 0), (-1, -1), 0),
            ('RIGHTPADDING', (0, 0), (-1, -1), 0),
            ('TOPPADDING', (0, 0), (-1, -1), 0),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 0),
        ]))
        return create_card_content([table], 7*inch)

    def payee_info(data, styles):
        content = [
            Paragraph(f"Account Holder: {data['account_holder']}", styles["bold"]),
            Paragraph(data['account_holder_address'], styles["normal"]),
            Paragraph(f"Account Type: {data['account_type']}", styles["normal"]),
            Paragraph(f"Statement Date: {data['statement_date']}", styles["normal"])
        ]
        return create_card_content(content, 7*inch)

    def checking_summary(data, styles):
        summary_data = [
            ["Beginning Balance", data['summary']['beginning_balance']],
            ["Deposits and Credits", f"{data['summary']['deposits_count']} items", data['summary']['deposits_total']],
            ["Withdrawals and Debits", f"{data['summary']['withdrawals_count']} items", data['summary']['withdrawals_total']],
            ["Total Transactions", f"{data['summary']['transactions_count']} items", ""],
            ["Ending Balance", "", data['summary']['ending_balance']]
        ]
        table = Table(summary_data, colWidths=[3*inch, 1.5*inch, 2*inch])
        table.setStyle(TableStyle([
            ('FONT', (0, 0), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('TEXTCOLOR', (0, 0), (-1, -1), colors.HexColor("#1f2937")),
            ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('LINEBELOW', (0, -1), (-1, -1), 1, colors.HexColor("#003087")),
            ('FONT', (0, -1), (-1, -1), 'Helvetica-Bold'),
        ]))
        return create_card_content([Paragraph("Checking Summary", styles["title"]), table], 7*inch)

    def important_account_info(data, styles):
        content = [Paragraph("Important Account Information", styles["title"])]
        if data['show_fee_waiver']:
            content.append(Paragraph("Monthly service fee waived due to qualifying activity.", styles["green_text"]))
        else:
            content.append(Paragraph("Monthly service fee applied. Maintain qualifying balance to waive fees.", styles["red_text"]))
        return create_card_content(content, 7*inch)

    def transactions(data, styles):
        transaction_data = [["Date", "Description", "Amount"]]
        for transaction in data['deposits'] + data['withdrawals']:
            style = styles["green_text"] if transaction['amount'] >= 0 else styles["red_text"]
            transaction_data.append([
                Paragraph(transaction['date'], styles["normal"]),
                Paragraph(transaction['description'], styles["normal"]),
                Paragraph(f"${abs(transaction['amount']):,.2f}", style)
            ])
        table = Table(transaction_data, colWidths=[1.5*inch, 3.5*inch, 1.5*inch])
        table.setStyle(TableStyle([
            ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONT', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor("#003087")),
            ('ALIGN', (2, 0), (2, -1), 'RIGHT'),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('LINEBELOW', (0, 0), (-1, 0), 1, colors.HexColor("#003087")),
        ]))
        return create_card_content([Paragraph("Transactions", styles["title"]), table], 7*inch)

    def daily_balance(data, styles):
        balance_data = [["Date", "Description", "Balance"]]
        for balance in data['daily_balances']:
            balance_data.append([
                Paragraph(balance['date'], styles["normal"]),
                Paragraph(balance['description'], styles["normal"]),
                Paragraph(f"${balance['amount']:,.2f}", styles["normal"])
            ])
        table = Table(balance_data, colWidths=[1.5*inch, 3.5*inch, 1.5*inch])
        table.setStyle(TableStyle([
            ('FONT', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONT', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor("#003087")),
            ('ALIGN', (2, 0), (2, -1), 'RIGHT'),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('LINEBELOW', (0, 0), (-1, 0), 1, colors.HexColor("#003087")),
        ]))
        return create_card_content([Paragraph("Daily Balances", styles["title"]), table], 7*inch)

    def footnotes(data, styles):
        content = [
            Paragraph("Footnotes", styles["title"]),
            Paragraph("Please review your statement for accuracy. Contact us if you have any questions.", styles["small"]),
            Paragraph("All transactions are subject to verification and may be adjusted.", styles["small"])
        ]
        return create_card_content(content, 7*inch)

    output_file = os.path.join(output_dir, f"chase_statement_app_{ctx['account_number_last_four']}.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch, leftMargin=0.5*inch, rightMargin=0.5*inch)
    styles = get_styles()
    elements = []

    layout_config = {
        "header_type": "detailed",
        "sections": ["header", "payee_info", "checking_summary", "important_account_info", "transactions", "daily_balance", "footnotes"]
    }
    section_functions = {
        "header": {"detailed": header_detailed},
        "payee_info": payee_info,
        "checking_summary": checking_summary,
        "important_account_info": important_account_info,
        "transactions": transactions,
        "daily_balance": daily_balance,
        "footnotes": footnotes
    }

    for section in layout_config["sections"]:
        if section == "header":
            header_func = section_functions["header"][layout_config["header_type"]]
            elements.append(header_func(ctx, styles))
        else:
            elements.append(section_functions[section](ctx, styles))
        elements.append(Spacer(1, 0.2*inch))

    doc.build(elements)
    print(f"PDF generated: {output_file}")


In [8]:

# ---------------------------------------------------------------------------------
# Generate all three statements
# ---------------------------------------------------------------------------------

if __name__ == "__main__":
    create_classic_statement(ctx)
    create_alternate_statement(ctx)
    create_app_statement(ctx)

PDF generated: out\chase_statement_classic.pdf
PDF generated: out\chase_statement_alternate.pdf
PDF generated: out\chase_statement_app_6611.pdf
